In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("heithemxnoor").project("ocr-ogjqm")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 251.9 kB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ocr-1 in folder:: 100%|██████████| 1910/1910 [00:00<00:00, 6845.61it/s]


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
train_dir = '/content/ocr-1/train'
val_dir = '/content/ocr-1/valid'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the data from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='sparse',
    color_mode='grayscale',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(28, 28),
    batch_size=32,
    class_mode='sparse',
    color_mode='grayscale',
    shuffle=False 
)

Found 1818 images belonging to 12 classes.
Found 65 images belonging to 12 classes.


In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)), 


    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(12, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


model.summary()


Argument `input_shape` is deprecated. Use `shape` instead.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 12)                  │           1,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 241,804 (944.55 KB)

 Trainable params: 241,804 (944.55 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.3165 - loss: 1.9998 - val_accuracy: 0.9688 - val_loss: 0.1608
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9062 - loss: 0.2281 - val_accuracy: 1.0000 - val_loss: 0.0094
Epoch 3/50


Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9458 - loss: 0.1872 - val_accuracy: 0.9688 - val_loss: 0.0761
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 336us/step - accuracy: 1.0000 - loss: 0.0503 - val_accuracy: 1.0000 - val_loss: 7.1857e-04
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9837 - loss: 0.0611 - val_accuracy: 0.9688 - val_loss: 0.0403
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 241us/step - accuracy: 0.9375 - loss: 0.1471 - val_accuracy: 1.0000 - val_loss: 9.2249e-04
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9914 - loss: 0.0285 - val_accuracy: 1.0000 - val_loss: 0.0067


In [ ]:
val_loss, val_acc = model.evaluate(val_generator)
print(f'Validation accuracy: {val_acc}')


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 3.3079e-05 
Validation accuracy: 1.0


In [ ]:
model.save('ocr_model.h5')

In [ ]:
class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','nt','tn']

img = Image.open("/content/tn.png") 
img = img.resize((28, 28))
img = img.convert('L')
img = tf.keras.utils.img_to_array(img) 
img = img.reshape((1, 28, 28, 1))
img = img / 255.0 

result = model.predict(img)
predicted_class = np.argmax(result, axis=1)[0]

img_to_display = img.squeeze()  


plt.imshow(img_to_display, cmap='gray') 
plt.title(f'Predicted Class: {class_labels[predicted_class]}') 
plt.axis('off') 
plt.show()

print(f'Predicted class: {class_labels[predicted_class]}')